# **Giới thiệu**

Trong bài này, chúng ta xây dựng một **pipeline học máy hoàn chỉnh** cho bài toán phân loại bệnh tim.  
Pipeline bao gồm các bước: EDA, tiền xử lý dữ liệu, trích xuất & lựa chọn đặc trưng, huấn luyện nhiều mô hình, đánh giá theo các chỉ số (Accuracy, Precision, Recall, F1-score), và trực quan hóa kết quả.  

Bộ dữ liệu sử dụng được lấy từ Kaggle:  
- [Heart Disease Dataset – oktayrdeki](https://www.kaggle.com/datasets/oktayrdeki/heart-disease)  

Toàn bộ nội dung cùng các Assignment khác trong môn **Machine Learning – HK251** được tổng hợp tại:  
- [Trang tổng hợp Machine Learning – Nhóm DNA05](https://nhinguyen140809.github.io/ml-asm-dna05/)  

Dưới đây là phần cài đặt và import các thư viện cần thiết để bắt đầu thực nghiệm.


# **Cài đặt và Import Thư viện**
Trong bước này, chúng ta sẽ:
- Cài đặt các thư viện cần thiết cho pipeline học máy.
- Import đầy đủ các module phục vụ cho **EDA, tiền xử lý, trích xuất đặc trưng, huấn luyện và đánh giá mô hình**.
- Thiết lập một số cấu hình cơ bản (ví dụ: tắt cảnh báo không quan trọng để log gọn gàng hơn).

Lưu ý:
- Người dùng có thể thêm/bớt thư viện nếu mở rộng pipeline (ví dụ: deep learning)
- Nếu notebook đã có sẵn môi trường, có thể bỏ qua lệnh `!pip install`.

In [ ]:
# ============================================================
# Cài đặt thư viện cần thiết
# ============================================================
!pip install kagglehub plotly scikit-learn imbalanced-learn -q

# ============================================================
# Import các thư viện
# ============================================================

# Xử lý hệ thống & dữ liệu
import os
import warnings
import numpy as np
import pandas as pd

# Trực quan hóa
import plotly.express as px
import matplotlib.pyplot as plt

# Tiền xử lý dữ liệu
from sklearn.preprocessing import (
    OrdinalEncoder, LabelEncoder,
    StandardScaler, MinMaxScaler
)
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV

# Mô hình học máy
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Pipeline & Feature Extraction
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

# Đánh giá mô hình
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report, confusion_matrix
)

# Xử lý mất cân bằng dữ liệu
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Hỗ trợ hiển thị
from IPython.display import display

# Kaggle dataset downloader
import kagglehub

# Thiết lập hiển thị & warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.3f}".format)

# Dùng modules
!git clone -q https://github.com/nhinguyen140809/ml-asm-dna05.git
import sys
sys.path.append("ml-asm-dna05/modules")
from ml_pipeline import search_best_model

print("Import và cài đặt thư viện thành công!")

# **Tải Dataset**

Trong bước này, chúng ta sẽ:
- Tải **Heart Disease Dataset** từ Kaggle thông qua `kagglehub`.
- Tự động tìm file `.csv` trong thư mục tải về và đọc vào `pandas.DataFrame`.
- Thiết lập cách xử lý giá trị thiếu (`na_values`) để dữ liệu được chuẩn hóa ngay từ đầu.

Cấu hình có thể thay đổi:
- `DATASET_NAME`: Tên dataset trên Kaggle (có thể thay đổi sang dataset khác).
- `FILE_EXTENSION`: Phần mở rộng của file dữ liệu cần đọc (mặc định `.csv`).
- `NA_VALUES`: Các giá trị coi là missing (có thể thêm `"NA"`, `"?"` nếu dataset khác).

In [ ]:
# ============================================================
# Cấu hình tải dataset
# ============================================================
CONFIG = {
    "DATASET_NAME": "oktayrdeki/heart-disease",  # Thay đổi nếu muốn dùng dataset khác
    "FILE_EXTENSION": ".csv",                    # Có thể đổi sang .xlsx nếu cần
    "NA_VALUES": ["", np.nan]                    # Các giá trị coi là missing
}

# ============================================================
# Hàm tải và đọc dataset
# ============================================================
def download_dataset(dataset_name: str, file_extension: str = ".csv") -> str:
    """
    Tải dataset từ Kaggle và trả về đường dẫn file dữ liệu.

    Args:
        dataset_name (str): Tên dataset trên Kaggle (ví dụ: "oktayrdeki/heart-disease")
        file_extension (str): Loại file cần lấy (.csv, .xlsx,...)

    Returns:
        str: Đường dẫn file dữ liệu
    """
    path = kagglehub.dataset_download(dataset_name)
    print(f"Dataset downloaded to: {path}")

    for f in os.listdir(path):
        if f.endswith(file_extension):
            return os.path.join(path, f)

    raise FileNotFoundError(f"Không tìm thấy file {file_extension} trong dataset!")

# ============================================================
# Đọc dữ liệu vào DataFrame
# ============================================================
csv_path = download_dataset(CONFIG["DATASET_NAME"], CONFIG["FILE_EXTENSION"])

df = pd.read_csv(
    csv_path,
    keep_default_na=False,
    na_values=CONFIG["NA_VALUES"]
)

print("Dataset loaded thành công!")
display(df.head())


# **1. EDA – Exploratory Data Analysis**


## 1.1. Tổng quan dữ liệu  

Trong bước này, chúng ta sẽ:  
- Hiển thị **5 dòng đầu tiên** để có cái nhìn nhanh về cấu trúc dữ liệu.  
- Tạo bảng **tổng quan dataset**: kiểu dữ liệu, số lượng non-null, số lượng null, số giá trị unique.  
- Thống kê **missing values** cho từng cột.  
- Sinh **summary statistics** riêng cho **numeric** và **categorical** columns.  

Cấu hình có thể thay đổi:
- `N_HEAD`: số dòng muốn hiển thị ở đầu dataset.  
- Có thể bật/tắt hiển thị từng bảng bằng các flag `SHOW_NUMERIC_SUMMARY`, `SHOW_CATEGORICAL_SUMMARY`.  

In [ ]:
# ============================================================
# Cấu hình EDA
# ============================================================
EDA_CONFIG = {
    "N_HEAD": 5,                     # Số dòng đầu hiển thị
    "SHOW_NUMERIC_SUMMARY": True,    # Bật/tắt thống kê numeric
    "SHOW_CATEGORICAL_SUMMARY": True # Bật/tắt thống kê categorical
}

# ============================================================
# Hiển thị 5 dòng đầu tiên
# ============================================================
display(df.head(EDA_CONFIG["N_HEAD"]).style.set_caption(f"{EDA_CONFIG['N_HEAD']} dòng đầu tiên"))
print("\n\n")

# ============================================================
# Tổng quan dataset
# ============================================================
overview = pd.DataFrame({
    "Dtype": df.dtypes,
    "Non-Null Count": df.notnull().sum(),
    "Null Count": df.isnull().sum(),
    "Unique Values": df.nunique()
})
display(overview.style.set_caption("Tổng quan Dataset"))
print("\n\n")

# ============================================================
# Thống kê Missing Values
# ============================================================
missing = df.isnull().sum().sort_values(ascending=False)
if missing.sum() > 0:
    display(missing.to_frame("Số lượng missing").style.set_caption("Thống kê Missing Values"))
else:
    print("Không có missing values trong dataset.")
print("\n\n")

# ============================================================
# Numeric summary
# ============================================================
if EDA_CONFIG["SHOW_NUMERIC_SUMMARY"]:
    num_summary = df.describe().T
    if not num_summary.empty:
        display(num_summary.style.set_caption("Thống kê Numeric Columns"))
print("\n\n")

# ============================================================
# Categorical summary
# ============================================================
if EDA_CONFIG["SHOW_CATEGORICAL_SUMMARY"]:
    cat_summary = df.describe(include=['object', 'category']).T
    if not cat_summary.empty:
        display(cat_summary.style.set_caption("Thống kê Categorical Columns"))


## **1.2. Thống kê tần suất nhãn**

Ở bước này, ta sẽ:  
- Đếm tần suất xuất hiện của từng nhãn trong cột **`Heart Disease Status`**.  
- Trực quan hoá bằng **biểu đồ cột** (bar chart).  

Cấu hình có thể thay đổi:  
- `LABEL_COL`: tên cột nhãn cần thống kê.  
- `BAR_COLOR`: chọn palette hiển thị (nếu muốn đồng bộ màu sắc).  
- `SHOW_PERCENTAGE`: hiển thị tỷ lệ % thay vì/together với số lượng tuyệt đối.  


In [ ]:
# ============================================================
# Cấu hình thống kê nhãn
# ============================================================
LABEL_CONFIG = {
    "LABEL_COL": "Heart Disease Status",  # Tên cột nhãn
    "SHOW_PERCENTAGE": True               # Hiển thị thêm % trên biểu đồ
}

# ============================================================
# Thống kê tần suất nhãn
# ============================================================
label_col = LABEL_CONFIG["LABEL_COL"]

label_counts = df[label_col].value_counts().reset_index()
label_counts.columns = [label_col, "Count"]

# Nếu cần hiển thị thêm %:
if LABEL_CONFIG["SHOW_PERCENTAGE"]:
    total = label_counts["Count"].sum()
    label_counts["Percentage"] = (label_counts["Count"] / total * 100).round(2)

# ============================================================
# Biểu đồ tần suất nhãn
# ============================================================
fig = px.bar(
    label_counts,
    x=label_col,
    y="Count",
    text="Count" if not LABEL_CONFIG["SHOW_PERCENTAGE"] else label_counts.apply(
        lambda r: f"{r['Count']} ({r['Percentage']}%)", axis=1
    ),
    color=label_col,
    title=f"Thống kê tần suất nhãn: {label_col}"
)

fig.update_traces(textposition="outside")
fig.update_layout(showlegend=False, xaxis_title=label_col, yaxis_title="Count")
fig.show()


## 1.3. Thống kê mô tả cho Numeric Columns  

Ở bước này ta sẽ:  
- Lọc ra các cột **numeric**.
- Tạo bảng thống kê mô tả (mean, std, min, max, quartiles).  
- Trực quan hoá phân phối bằng **histogram kèm boxplot** để phát hiện phân phối và outliers.  

Cấu hình có thể thay đổi:  
- `NUMERIC_BINS`: số lượng bins trong histogram.  
- `SHOW_BOXPLOT`: có hiển thị boxplot phụ không.  
- `HIST_COLOR`: màu histogram.  


In [ ]:
# ============================================================
# Cấu hình thống kê numeric
# ============================================================
NUMERIC_CONFIG = {
    "NUMERIC_BINS": 30,            # Số bins trong histogram
    "SHOW_BOXPLOT": True,          # Hiển thị boxplot trên histogram
    "HIST_COLOR": "dodgerblue"     # Màu sắc histogram
}

# ============================================================
# Lọc numeric columns
# ============================================================
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# ============================================================
# Thống kê mô tả
# ============================================================
if numeric_cols:
    display(df[numeric_cols].describe().T.style.set_caption("Thống kê Numeric Columns"))
else:
    print("Không có numeric columns trong dataset.")

# ============================================================
# Trực quan phân phối numeric
# ============================================================
for col in numeric_cols:
    fig = px.histogram(
        df,
        x=col,
        nbins=NUMERIC_CONFIG["NUMERIC_BINS"],
        title=f"Phân phối {col}",
        marginal="box" if NUMERIC_CONFIG["SHOW_BOXPLOT"] else None,
        color_discrete_sequence=[NUMERIC_CONFIG["HIST_COLOR"]]
    )
    fig.update_layout(yaxis_title="Số lượng mẫu", bargap=0.05)
    fig.show()


## 1.4. Thống kê mô tả cho Categorical Columns  

Ở bước này ta sẽ:  
- Lọc ra các **categorical columns** (ngoại trừ cột target `Heart Disease Status`).  
- Tính tần suất xuất hiện của từng giá trị.  
- Trực quan hoá bằng **pie chart** để thấy cơ cấu phân bố.  

Cấu hình có thể thay đổi:  
- `SHOW_LIMIT`: số lượng category tối đa sẽ hiển thị (giúp tránh pie chart quá rối với nhiều nhãn).  
- `PIE_TEXTINFO`: dạng thông tin hiển thị trên chart (`"percent"`, `"value"`, `"percent+value"`, …).  
- `PIE_HOLE`: hệ số donut (0 = pie chart thường, >0 = donut chart).  


In [ ]:
# ============================================================
# Cấu hình thống kê categorical
# ============================================================
CATEGORICAL_CONFIG = {
    "SHOW_LIMIT": 10,                   # Hiển thị tối đa bao nhiêu category (None = tất cả)
    "PIE_TEXTINFO": "percent+value",    # Hiển thị % và số lượng
    "PIE_HOLE": 0.0,                    # 0 = pie chart thường, >0 = donut chart
    "LABEL_COL": "Heart Disease Status" # Tên cột nhãn
}

# ============================================================
# Lọc categorical columns (bỏ target)
# ============================================================
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col != LABEL_CONFIG["LABEL_COL"]]

# ============================================================
# Thống kê và trực quan hóa categorical
# ============================================================
if categorical_cols:
    for col in categorical_cols:
        counts = df[col].value_counts().reset_index()
        counts.columns = [col, "Count"]

        # Giới hạn số lượng category hiển thị
        if CATEGORICAL_CONFIG["SHOW_LIMIT"] is not None:
            counts = counts.head(CATEGORICAL_CONFIG["SHOW_LIMIT"])

        fig = px.pie(
            counts,
            names=col,
            values="Count",
            title=f"Cơ cấu {col}",
            hole=CATEGORICAL_CONFIG["PIE_HOLE"]
        )
        fig.update_traces(textinfo=CATEGORICAL_CONFIG["PIE_TEXTINFO"])
        fig.show()
else:
    print("Không có categorical columns trong dataset.")


# **2. Tiền xử lý dữ liệu**


## 2.1. Encoding  

Ở bước này ta cần:  
1. **Định nghĩa thứ tự cho các biến categorical** (Ordinal Encoding).  
   - Một số biến có mức độ (`Low < Medium < High`).  
   - Một số biến boolean (`Yes/No`).  
   - Giới tính (`Male/Female`)
2. **Áp dụng `OrdinalEncoder`** cho các cột categorical.  
   - Bỏ qua giá trị missing (sẽ xử lý sau bằng KNN Imputer).  
3. **Encode target label** (`Heart Disease Status`) bằng `LabelEncoder`.  

Cấu hình có thể thay đổi:  
- `CATEGORY_MAP`: định nghĩa thứ tự cho từng nhóm biến.  
- `TARGET_COLUMN`: tên cột target.  
- `USE_ONEHOT_FOR`: nếu muốn OneHot thay vì Ordinal cho một số biến (vd: Gender).  


In [ ]:
# ============================================================
# Cấu hình encoding
# ============================================================
CATEGORY_MAP = {
    "Exercise Habits": ["Low", "Medium", "High"],
    "Stress Level": ["Low", "Medium", "High"],
    "Sugar Consumption": ["Low", "Medium", "High"],
    "Alcohol Consumption": ["None", "Low", "Medium", "High"],
    "Gender": ["Male", "Female"],
    # Các cột boolean khác (No/Yes)
}
DEFAULT_BOOLEAN = ["No", "Yes"]

TARGET_COLUMN = "Heart Disease Status"

# ============================================================
# Build categories list cho OrdinalEncoder
# ============================================================
categories = []
for col in categorical_cols:
    if col in CATEGORY_MAP:
        categories.append(CATEGORY_MAP[col])
    else:
        categories.append(DEFAULT_BOOLEAN)

# ============================================================
# Encode categorical columns bằng OrdinalEncoder
# ============================================================
for col, cats in zip(categorical_cols, categories):
    ordinal_encoder = OrdinalEncoder(categories=[cats])
    mask = df[col].notna()   # Bỏ qua missing
    df.loc[mask, col] = ordinal_encoder.fit_transform(df.loc[mask, [col]])

# ============================================================
# Encode target bằng LabelEncoder
# ============================================================
label_encoder = LabelEncoder()
df[TARGET_COLUMN] = label_encoder.fit_transform(df[TARGET_COLUMN])

# ============================================================
# Kiểm tra dữ liệu sau khi mã hoá
# ============================================================
display(df.head().style.set_caption("Dataset sau khi Encoding"))


## 2.2. Chia tập Train - Test  

Ở bước này ta sẽ:  
- Tách **features (X)** và **target (y)**.  
- Chia dữ liệu thành **train/test** để huấn luyện và đánh giá mô hình.  
- Dùng `stratify=y` để giữ phân phối nhãn đồng đều giữa train/test.  

Cấu hình có thể thay đổi:  
- `TARGET_COLUMN`: tên cột targ_


In [ ]:
# ============================================================
# Cấu hình train-test split
# ============================================================
TARGET_COLUMN = "Heart Disease Status"
TEST_SIZE = 0.2
RANDOM_STATE = 42

# ============================================================
# Tách features và target
# ============================================================
X = df.drop(columns=[TARGET_COLUMN])
y = df[TARGET_COLUMN]

# ============================================================
# Chia train - test
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

# ============================================================
# Kiểm tra kết quả
# ============================================================
print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")
print("\n\n")
display(y_train.value_counts().to_frame("Count").style.set_caption("Label Distribution - Train"))
print("\n\n")
display(y_test.value_counts().to_frame("Count").style.set_caption("Label Distribution - Test"))

# Xem vài dòng dữ liệu train
display(X_train.head().style.set_caption("5 dòng đầu tiên - Train set"))
print("\n\n")

# Xem vài dòng dữ liệu test
display(X_test.head().style.set_caption("5 dòng đầu tiên - Test set"))

## 2.3. Xử lý giá trị thiếu  

Ở bước này ta sẽ:  
- Dùng **KNNImputer** để điền giá trị thiếu dựa trên lân cận gần nhất.  
- Sau khi Impute, ép kiểu các cột categorical về **int** (vì KNN sinh ra float).  
- Kiểm tra lại xem còn missing value không.  

In [ ]:
# ============================================================
# KNN Imputer cho dữ liệu thiếu
# ============================================================
knn_imputer = KNNImputer(n_neighbors=5)

# Fit + transform train
X_train_imputed = pd.DataFrame(
    knn_imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

# Transform test (dùng cùng imputer để tránh data leakage)
X_test_imputed = pd.DataFrame(
    knn_imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# ============================================================
# Ép categorical về int (do KNN sinh float)
# ============================================================
for col in categorical_cols:
    X_train_imputed[col] = X_train_imputed[col].round().astype("Int64")
    X_test_imputed[col] = X_test_imputed[col].round().astype("Int64")

# ============================================================
# Kiểm tra còn missing value không
# ============================================================
def check_missing(df, name=""):
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if not missing.empty:
        display(missing.to_frame("Số lượng missing").style.set_caption(f"Missing Values - {name}"))
    else:
        print(f"Không còn missing trong {name}")

check_missing(X_train_imputed, "Train set")
check_missing(X_test_imputed, "Test set")

# Xem vài dòng dữ liệu sau khi xử lý
display(X_train_imputed.head().style.set_caption("5 dòng đầu tiên sau khi Impute - Train"))


## 2.4. Scaling  

- Mục đích: Chuẩn hoá các feature về cùng thang đo để mô hình học hiệu quả hơn.  
- Kỹ thuật sử dụng mặc định: `MinMaxScaler(feature_range=(0, 1))`  
  - Bạn có thể thay đổi cấu hình `feature_range`

- Có thể thay thế bằng: `StandardScaler`, `RobustScaler`,...

In [ ]:
# ============================================================
# Lựa chọn scaler (có thể đổi sang StandardScaler, RobustScaler)
# ============================================================
scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = StandardScaler()
# scaler = RobustScaler()

# ============================================================
# Fit & transform
# ============================================================
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_imputed),
    columns=X_train_imputed.columns,
    index=X_train_imputed.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_imputed),
    columns=X_test_imputed.columns,
    index=X_test_imputed.index
)

# ============================================================
# Kiểm tra dữ liệu sau scaling
# ============================================================
display(X_train_scaled.head().style.set_caption("5 dòng đầu tiên sau khi Scaling - Train"))


## 2.5. Xử lý mất cân bằng dữ liệu  

- **Vấn đề:** Dữ liệu nhãn mất cân bằng khiến mô hình dễ nghiêng về lớp chiếm đa số.  

- **Kỹ thuật áp dụng:**  
  - **SMOTE (Synthetic Minority Oversampling Technique)**: sinh thêm mẫu giả lập cho lớp thiểu số.  

- **Cấu hình quan trọng:**  
  - `sampling_strategy`: tỉ lệ giữa lớp nhỏ và lớp lớn sau oversampling.  
    - `0.5` nghĩa là lớp nhỏ = 50% lớp lớn.  
    - `1.0` nghĩa là cân bằng tuyệt đối.  
  - `k_neighbors`: số hàng xóm dùng để sinh mẫu mới.  
  - `random_state`: để tái lập kết quả.  


In [ ]:
# ============================================================
# Cấu hình SMOTE
# ============================================================
smote = SMOTE(
    sampling_strategy=0.5,  # lớp thiểu số = 50% lớp đa số
    k_neighbors=5,
    random_state=42
)

# ============================================================
# Resampling (chỉ áp dụng trên tập train)
# ============================================================
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# ============================================================
# Kiểm tra phân phối nhãn trước & sau
# ============================================================
display(y_train.value_counts().to_frame("Count").style.set_caption("Trước khi xử lý mất cân bằng"))
print("\n\n")
display(y_train_res.value_counts().to_frame("Count").style.set_caption("Sau khi xử lý mất cân bằng"))

# **3. Trích xuất & lựa chọn đặc trưng**

- Trong bài toán này, ngoài việc sử dụng toàn bộ tập đặc trưng gốc, chúng ta có thể áp dụng các kỹ thuật **giảm chiều & trích xuất đặc trưng** nhằm:  
  - Loại bỏ nhiễu và thông tin dư thừa.  
  - Giảm chi phí tính toán, tăng tốc huấn luyện.  
  - Tránh hiện tượng **curse of dimensionality** (khi số chiều quá lớn).  

- **Kỹ thuật áp dụng:**  
  - **PCA (Principal Component Analysis):**
  - PCA sẽ được cấu hình như một bước tùy chọn trong pipeline huấn luyện (ở mục 4.1).  
  - Người dùng có thể bật/tắt hoặc thay đổi số thành phần PCA (`n_components`) để đánh giá ảnh hưởng đến hiệu năng mô hình.  


# **4. Học máy**


## 4.1. Kiểm tra tập train - test  
- Sau các bước **tiền xử lý (impute, scaling, xử lý mất cân bằng)**, dữ liệu đã sẵn sàng để huấn luyện.  
- Ở bước này, ta cần:  
  1. Xác nhận kích thước tập train/test.  
  2. Kiểm tra phân bố nhãn (label distribution) để đảm bảo tính cân bằng sau khi áp dụng **SMOTE**.  
  3. Xem trước vài dòng dữ liệu để đảm bảo dữ liệu đã được xử lý đúng (không còn missing, đúng kiểu dữ liệu).  

In [ ]:
# Gán lại tập train/test sau xử lý
X_train = X_train_res
y_train = y_train_res

X_test = X_test_scaled
y_test = y_test

# Convert về DataFrame để dễ trực quan
X_train_df = pd.DataFrame(X_train, columns=df.columns[:-1])  # bỏ cột target
y_train_df = pd.Series(y_train, name='Heart Disease Status')

X_test_df = pd.DataFrame(X_test, columns=df.columns[:-1])
y_test_df = pd.Series(y_test, name='Heart Disease Status')

# Thông tin tổng quan
print(f"Train samples: {len(X_train_df)}, Test samples: {len(X_test_df)}\n")

print("Train label distribution:")
display(
    y_train_df.value_counts()
    .to_frame()
    .rename(columns={'Heart Disease Status':'Count'})
)

print("\nTest label distribution:")
display(
    y_test_df.value_counts()
    .to_frame()
    .rename(columns={'Heart Disease Status':'Count'})
)

# Hiển thị vài dòng đầu
print("\nSample rows of X_train:")
display(X_train_df.head())

## 4.2. Tìm tham số tối ưu cho từng mô hình

- **Mục tiêu:**  
  Tối ưu **Recall** để giảm bỏ sót ca bệnh (False Negative).

- **Chiến lược:**  
  1. Dùng `GridSearchCV` để tìm tham số tốt nhất cho từng mô hình.  
  2. Thêm **PCA** vào pipeline **chỉ khi param_grid có `pca__*`**.  
  3. Bật/tắt tìm kiếm bằng `SEARCH_FOR_BEST`:
     - `False` → bỏ qua GridSearchCV, tiết kiệm thời gian.  
     - `True` → chạy tìm tham số cho các model được chọn.  
  4. Chỉ số tối ưu thay đổi bằng `SCORING_METRIC` (`"recall"`, `"precision"`, `"f1"`, `"accuracy"`).

- **Cấu hình:**  
  - Định nghĩa `models_dict` gồm model và `param_grid` tương ứng.  
  - Chọn model cần tìm tham số bằng danh sách `models_to_search`.  

- **Lưu ý:**  
  - Kết quả chỉ mang tính **tham khảo**, do GridSearchCV dùng cross-validation trên train set. Tham số tìm được chưa chắc tối ưu trên test set.  
  - Có thể thay đổi `SCORING_METRIC`, `models_to_search`, và cấu hình trong `models_dict` tùy yêu cầu bài toán.


In [ ]:
SEARCH_FOR_BEST = False  # Đặt True để chạy GridSearchCV (có thể tốn thời gian)
SCORING_METRIC = "recall"  # Metric ưu tiên

# ==========================
# Danh sách model + param_grid
# ==========================
models_dict = {
    "RandomForest": [
        RandomForestClassifier(random_state=42),
        {
            'pca__n_components': [0.7, 0.9, 0.99],
            'clf__n_estimators': [50, 100, 200],
            'clf__max_depth': [20, 50],
            'clf__min_samples_split': [2, 5, 10],
        }
    ],
    "LogisticRegression": [
        LogisticRegression(max_iter=3000, random_state=42),
        {
            'pca__n_components': [0.7, 0.8, 0.9, 0.99],
            'clf__C': [0.01, 0.1, 1, 10, 100],
            'clf__penalty': ["l1", "l2"],
            'clf__solver': ["liblinear", "saga"],
            'clf__class_weight': [None, "balanced"]
        }
    ],
    "SVM": [
        SVC(probability=True, random_state=42, max_iter=5000),
        {
            'pca__n_components': [0.7, 0.8, 0.9, 0.99],
            'clf__C': [0.1, 1, 10, 100],
            'clf__kernel': ["linear", "rbf", "poly"],
            'clf__class_weight': [None, "balanced"]
        }
    ],
    "KNN": [
        KNeighborsClassifier(),
        {
            'pca__n_components': [0.7, 0.8, 0.9, 0.99],
            'clf__n_neighbors': [3, 5, 7, 9, 11, 15],
            'clf__weights': ["uniform", "distance"],
            'clf__p': [1, 2],
            'clf__metric': ["minkowski"]
        }
    ],
    "DecisionTree": [
        DecisionTreeClassifier(random_state=42),
        {
            'pca__n_components': [0.7, 0.8, 0.9, 0.99],
            'clf__criterion': ["gini", "entropy"],
            'clf__max_depth': [None, 5, 10, 20],
            'clf__min_samples_split': [2, 5],
            'clf__class_weight': [None, "balanced"],
            'clf__ccp_alpha': [0.0, 0.01]
        }
    ],
    "NaiveBayes": [
        GaussianNB(),
        {
            'pca__n_components': [0.7, 0.8, 0.9, 0.99],
            'clf__var_smoothing': [1e-09, 1e-06, 1e-03, 1]
        }
    ]
}

# ==========================
# Chọn model muốn search
# ==========================
models_to_search = ["RandomForest", "LogisticRegression", "DecisionTree"]
results = []

if SEARCH_FOR_BEST:
    for model_name in models_to_search:
        model, param_grid = models_dict[model_name]
        print(f"Running GridSearchCV for {model_name} ...")

        use_pca = any(key.startswith("pca__") for key in param_grid.keys())

        res = search_best_model( X_train, y_train, model=model, param_grid=param_grid, scoring_metric=SCORING_METRIC, use_pca=use_pca)

        results.append({"Model": model_name, "Best Params": res["best_params"], "Best Score": res["best_score"]})
else:
    print("Bỏ qua GridSearchCV (SEARCH_FOR_BEST=False)")

# ==========================
# Hiển thị kết quả
# ==========================
results_df = pd.DataFrame(results)
if not results_df.empty:
    display(results_df.style.hide(axis="index"))
else:
    print("Chưa có kết quả (SEARCH_FOR_BEST=False)")


## 4.3. Huấn luyện các mô hình với PCA  

- Sau khi tiền xử lý, ta huấn luyện nhiều mô hình học máy trên dữ liệu đã giảm chiều bằng **PCA**.  
- Các mức phương sai được giữ lại: **70%, 80%, 90%, 99%** và **None** (không có PCA).  
- Các mô hình sử dụng:  
  - Random Forest  
  - SVM  
  - Logistic Regression  
  - k-NN  
  - Naive Bayes  
  - Decision Tree  

### Quy trình  
1. Xây dựng **Pipeline** gồm PCA và mô hình.  
2. Huấn luyện trên tập **train** (đã cân bằng bằng SMOTE).  
3. Dự đoán trên tập **test** (không áp dụng SMOTE).  
4. Đánh giá mô hình theo các metric:  
   - **Accuracy**  
   - **Precision**  
   - **Recall**  
   - **F1-score**  
5. Hiển thị **Confusion Matrix** cho từng mô hình để quan sát chi tiết.  

### Lưu ý  
- Các cấu hình (siêu tham số) trong phần này được thiết lập dựa trên cấu hình tối ưu mà nhóm đã thử nghiệm ở **mục 4.2**.
- Có thể điều chỉnh lại cấu hình nếu muốn kiểm chứng hoặc cải thiện hiệu năng.  
- Kết quả tối ưu ở **mục 4.2** chỉ mang tính **tham khảo**, vì được tìm bằng **GridSearchCV** trên tập train và chưa chắc đã đạt hiệu quả tốt nhất trên tập test thực tế.  


In [ ]:
# Định nghĩa danh sách mô hình
models = {
    "Random Forest": RandomForestClassifier(
        max_depth=20, min_samples_split=2, n_estimators=200, class_weight="balanced"
    ),
    "SVM": SVC(C=10, kernel='rbf', class_weight="balanced"),
    "Logistic Regression": LogisticRegression(
        C=0.1, class_weight="balanced", penalty="l1", solver="liblinear"
    ),
    "KNN": KNeighborsClassifier(
        metric="minkowski", n_neighbors=3, p=1, weights='distance'
    ),
    "Naive Bayes": GaussianNB(var_smoothing=1e-09),
    "Decision Tree": DecisionTreeClassifier(
        ccp_alpha=0.0, criterion='gini', max_depth=5,
        min_samples_split=2, class_weight="balanced"
    ),
}


# Các mức PCA giữ lại, thêm None để không dùng PCA
pca_variances = [0.7, 0.8, 0.9, 0.99] # Thêm None nếu muốn

# Lưu kết quả
results = []

# Huấn luyện & đánh giá
for pca_var in pca_variances:
    for model_name, model in models.items():
        pca_label = "no PCA" if pca_var is None else f"{int(pca_var*100)}%"
        if pca_label == "no PCA":
          print(f"\n=== Training {model_name} without PCA ===")
        else:
          print(f"\n=== Training {model_name} with PCA={pca_label} ==")

        # Pipeline PCA + Model hoặc chỉ model nếu pca_var is None
        steps = []
        if pca_var is not None:
            steps.append(('pca', PCA(n_components=pca_var)))
        steps.append(('clf', model))
        pipeline = Pipeline(steps)

        # Train
        pipeline.fit(X_train, y_train)

        # Predict trên tập test
        y_pred = pipeline.predict(X_test)

        # Đánh giá
        acc  = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec  = recall_score(y_test, y_pred, zero_division=0)
        f1   = f1_score(y_test, y_pred, zero_division=0)

        results.append({
            "Model": model_name,
            "PCA": pca_label,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-score": f1
        })

        # Hiển thị Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        cm_df = pd.DataFrame(
            cm,
            index=[f"Actual {cls}" for cls in label_encoder.classes_],
            columns=[f"Pred {cls}" for cls in label_encoder.classes_]
        )
        display(cm_df)

## 4.4. So sánh và đánh giá

Sau khi huấn luyện các mô hình với các mức PCA khác nhau, ta tiến hành:  
  1. Tổng hợp kết quả đánh giá của tất cả mô hình.  
  2. Xác định mô hình tốt nhất theo từng metric (**Accuracy, Precision, Recall, F1-score**).  
  3. Trực quan hóa so sánh các mô hình dưới từng mức PCA để quan sát sự khác biệt.  

In [ ]:
# ======================
# 1. Tổng hợp kết quả
# ======================
results_df = pd.DataFrame(results)

print("=== Comparison Table ===")
display(results_df.style.hide(axis="index"))

# ======================
# 2. Mô hình tốt nhất theo từng metric
# ======================
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
best_summary = []

for metric in metrics:
    idx = results_df[metric].idxmax()
    best_row = results_df.loc[idx]
    best_summary.append({
        'Metric': metric,
        'Best Model': best_row['Model'],
        'Best PCA': best_row['PCA'],
        'Best Score': best_row[metric]
    })

print("\n=== Best Model per Metric ===")
display(pd.DataFrame(best_summary).style.hide(axis="index"))

# ======================
# 3. Trực quan hóa kết quả theo từng mức PCA
# ======================
pca_values = sorted(results_df['PCA'].unique())
bar_width = 0.2

for pca in pca_values:
    df_pca = results_df[results_df['PCA'] == pca]

    models_compare = df_pca['Model'].values
    accuracy = df_pca['Accuracy'].values
    precision = df_pca['Precision'].values
    recall = df_pca['Recall'].values
    f1_score_vals = df_pca['F1-score'].values

    x = np.arange(len(models_compare))

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(x - 1.5*bar_width, accuracy, bar_width, label="Accuracy", color="blue")
    ax.bar(x - 0.5*bar_width, precision, bar_width, label="Precision", color="green")
    ax.bar(x + 0.5*bar_width, recall, bar_width, label="Recall", color="orange")
    ax.bar(x + 1.5*bar_width, f1_score_vals, bar_width, label="F1-score", color="red")

    ax.set_xlabel("Models", fontsize=12)
    ax.set_ylabel("Score", fontsize=12)
    ax.set_title(f"Model Comparison for PCA = {pca}%", fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(models_compare, rotation=15)
    ax.set_ylim(0, 1)
    ax.legend()
    fig.tight_layout()
    plt.show()

# **5. Kết luận**

Với bộ dữ liệu **Heart Disease** từ Kaggle, nhóm đã xây dựng và thử nghiệm một pipeline phân loại bệnh tim với nhiều mô hình khác nhau, kết hợp PCA và các tiêu chí đánh giá phổ biến. Kết quả cho thấy từng mô hình có ưu thế riêng, đồng thời khẳng định **Recall** là chỉ số quan trọng trong bối cảnh y tế nhằm hạn chế bỏ sót ca bệnh.  

Tham khảo:  
- [Heart Disease Prediction with 83.8% Accuracy](https://www.kaggle.com/code/hossainhedayati/heart-disease-prediction-with-83-8-accuracy)
